In [ ]:
# Run the following two cells if you are using Google Colab
# Install Colab Anaconda (takes around 20 seconds)
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install esda(Exploratory Spatial Data Analysis) & libpysal (takes around 3 minutes)
!conda install --channel conda-forge esda --y

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/KHU_Urban_Geography/Lab4

## Social Area Analysis: calculating Moran's I and LISA with census data

This notebook demonstrates how to calculate Moran's I and Local Indicator of Spatial Association (LISA; aka Local Moran's I) based on census data. To be specific, it employs socioeconomic, ethnic, and age data, focusing on census tract-level data in Chicago, Illinois. 

### Data: 
* Census Tracts: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html <br>
* Social Vulnerability Index (SVI): https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documentation.pdf

<div>
<img src="https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI-Variables.png?_=02699" width="500"/>
</div>

### Steps:
1. Load census data and geometry data
2. Preprocessing data 
    * Merge (join) data 
    * Rename columns 
3. Analyze the spatial correlation of census data 
    * Calculate contiguity-baseed spatial weights
    * Calculate Moran's I
    * Calculate LISA
4. Visualize the results

## Import Packages

A Python package is a way of organizing related Python modules into a single directory hierarchy. It provides a mechanism for grouping Python code files, resources, and configuration settings in a structured manner, making it easier to manage and distribute code. They also facilitate code reuse and distribution by allowing developers to bundle related functionality together and share it with others.

The following packages are used in this notebook:<br>
`libpysal` is a Python package for spatial analysis. It provides foundational algorithms and data structures that support the rest of the library. <br>
source: https://pysal.org/libpysal/ <br>

`esda` is a Python package and implements methods for the analysis of both global (map-wide) and local (focal) spatial autocorrelation, for both continuous and binary data. In addition, the package increasingly offers cutting-edge statistics about boundary strength and measures of aggregation error in statistical analyses. <br>
source: https://pysal.org/esda/index.html <br>

In [ ]:
# New Packages
import libpysal
import esda

# Packages that investigated in the previous project
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Etc
import warnings
warnings.filterwarnings('ignore')

## 1. Load census data and geometry data

In [ ]:
!pwd

In [ ]:
# Load geometry data from the file
geom_gdf = gpd.read_file('./data/tracts_Cook_IL.geojson')
geom_gdf

In [ ]:
geom_gdf.plot()

In [ ]:
# Load socioeconomic data from the file
soc_df = pd.read_csv('./data/socioeconomic.csv')
soc_df

In [ ]:
soc_df.hist('EP_POV150', bins=30)

---
### *Exercise*
1. (7 points) Load `ethnic.csv` file into `ethnic_df` using the syntax below. You can find the necesary file in the data folder. 

```python
    ethnic_df = pd.read_csv(`FILE PATH`)
    ethnic_df.head(3)
```
---

In [ ]:
# Your code here
ethnic_df = pd.read_csv(`FILE PATH`)
ethnic_df.head(3)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert ethnic_df.shape == (1328, 5)

print('Success!')

---
### *Exercise*
2. (7 points) Load `age.csv` file into `age_df` using the syntax below. You can find the necesary file in the data folder. 

```python
    age_df = pd.read_csv(`FILE PATH`)
    age_df.head(3)
```
---

In [ ]:
# Your code here
age_df = pd.read_csv(`FILE PATH`)
age_df.head(3)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert age_df.shape == (1328, 2)

print('Success!')

## 2. Preprocessing data 
### 2.1. Merge (join) data

In [ ]:
# GeoDataFrame with geometry
geom_gdf.head(3)

In [ ]:
# Data Frame with socioeconomic data
soc_df.head(3)

In [ ]:
# You can conduct a join as shown below. 
# The following code will generate an error because the datatype of those two columns are different. 
# ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
soc_geom = geom_gdf.merge(soc_df, left_on='GEOID', right_on='FIPS')
soc_geom

Before you proceed to the merge step, you need to change the datatype of the 'FIPS' column in the 'soc_df' DataFrame to 'str'

In [ ]:
# Check the data types of the columns
geom_gdf.dtypes

In [ ]:
# Check the data types of the columns
soc_df.dtypes

In [ ]:
# Change the datatype of FIPS column into string
soc_df['FIPS'] = soc_df['FIPS'].astype(str)

In [ ]:
# Check the data types of the columns
soc_df.dtypes

In [ ]:
# Now you should be able to conduct a join. 
soc_gdf = geom_gdf.merge(soc_df, left_on='GEOID', right_on='FIPS')
soc_gdf

---
### *Exercise*

3. (7 points) Change the datatype of 'FIPS' column in both `ethnic_df` and `age_df` to string by using the syntax below. 

```python
    ethnic_df[`COLUMN NAME`] = ethnic_df[`COLUMN NAME`].astype(`DATA TYPE`)
    age_df[`COLUMN NAME`] = age_df[`COLUMN NAME`].astype(`DATA TYPE`)
```
---

In [ ]:
# Your code here
ethnic_df[`COLUMN NAME`] = ethnic_df[`COLUMN NAME`].astype(`DATA TYPE`)
age_df[`COLUMN NAME`] = age_df[`COLUMN NAME`].astype(`DATA TYPE`)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert ethnic_df['FIPS'].dtypes == 'object'
assert age_df['FIPS'].dtypes == 'object'

print('Success!')

---
### *Exercise*

4. (7 points) Join `geom_gdf` GeoDataFrame with `ethnic_df` based on an appropriate columns (Check the example above). Then, assign the result to `ethnic_gdf`. 

``` python
    ethnic_gdf = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)
    ethnic_gdf.head(3)
```
---

In [ ]:
# Your code here
ethnic_gdf = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)
ethnic_gdf.head(3)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert 'EP_WHITE' in ethnic_gdf.columns
assert 'geometry' in ethnic_gdf.columns

print('Success!')

---
### *Exercise*
5. (7 points) Join `geom_gdf` GeoDataFrame with `age_df` based on an appropriate columns (Check the example above). Then, assign the result to `age_gdf`. 

``` python
    age_gdf = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)
    age_gdf.head(3)
```
---

In [ ]:
# Your code here
age_gdf = `DATAFRAME A`.merge(`DATAFRAME B`, on=`COLUMN NAME`)
age_gdf.head(3)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert 'EP_AGE17' in age_gdf.columns
assert 'geometry' in age_gdf.columns

print('Success!')

If you want to merge multiple tables, you can conduct merge mupltiple times. 

```python
    merged_df = pd.merge(df1, df2, on='key') 
    # or 
    merged_df = df1.merge(df2, on='key')

    # and then, merge the result with another DataFrame
    merged_df = pd.merge(merged_df, df3, on='key')
```

In [ ]:
census_gdf = geom_gdf.merge(soc_df, left_on='GEOID', right_on='FIPS')
census_gdf = census_gdf.merge(ethnic_df, left_on='FIPS', right_on='FIPS')
census_gdf = census_gdf.merge(age_df, left_on='FIPS', right_on='FIPS')
census_gdf.head(3)

### 2.2. Rename columns

The name of the columns in the GeoDataFrame is not intuitive. You can rename the columns to make it easier to understand.


Syntax of renaming columns in GeoDataFrame / DataFrame:

```python
    gdf = gdf.rename(columns={'old_name_1': 'new_name_1',
                              'old_name_2': 'new_name_2',
                              'old_name_3': 'new_name_3'}
                              )
```

In [ ]:
census_gdf = census_gdf.rename(columns={'EP_POV150': 'POVERTY', 
                           'EP_WHITE': 'WHITE',
                            'EP_AFAM': 'BLACK',
                            'EP_HISP'  : 'HISPANIC',
                            'EP_ASIAN' : 'ASIAN',
                            'EP_AGE17' : 'YOUNG',
                           })
census_gdf.head(3)

## 3. Analyze the spatial correlation of census data 

Global **Moran's I** demonstrates how geographical phenomena are correlated over space, meaning whether closer things is more related than distant things. The method provides an index with the range -1 to 1; namely, -1 is a strong negative spatial autocorrelation and 1 is a strong positive spatial autocorrelation.
<br><br>
While Global Moran's I only provides one index to demonstrate spatial autocorrelation, **Local Indicator of Spatial Association (LISA)**, as known as Local Moran's I explains where high (i.e., HH Cluster) and low (LL Cluster) values are clustered.

<div>
<img src="https://geodacenter.github.io/workbook/6c_local_multi/pics6c/1_bivspacetime.png" width="700"/>
</div>


In [ ]:
# Visualize the data 

fig, axes = plt.subplots(1, 6, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(['POVERTY', 'YOUNG', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN']):
    census_gdf.plot(col, ax=axes[i], cmap='viridis', scheme='NaturalBreaks')
    axes[i].set_title(col)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

### 3.1. Calculate contiguity-baseed spatial weights

The following figure will demonstrate what **contiguity** means. Here, we use Queen's case. 

<div>
<img src="./image/contiguity.jpg" width="500"/>
</div>

In [ ]:
# Calculate spatial relationship (contiguity) of geometry
w = libpysal.weights.Queen.from_dataframe(census_gdf)
w

In [ ]:
w.neighbors

In [ ]:
census_gdf.loc[[0, ]]

In [ ]:
census_gdf.loc[[657, 818, 660, 661, 714, 764, 1166], ]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
census_gdf.loc[[0, ]].plot(ax=ax, color='black')
census_gdf.loc[[657, 818, 660, 661, 714, 764, 1166], ].boundary.plot(ax=ax, color='grey')

### 3.2. Calculate Moran's I

Syntax for Moran's I is as follows:

```python
    # Moran's I
    mi = esda.moran.Moran(`VARIABLE`, `SPATIAL WEIGHTS MATRIX`)
```

Variable `mi` has the following attributes:
* `I` : Moran's I value
* `p_sim` : p-value <br>

source: https://pysal.org/esda/generated/esda.Moran.html#esda.Moran

In [ ]:
# Calculate Moran's I for poverty
mi_pov = esda.Moran(census_gdf['POVERTY'], w)
mi_pov

In [ ]:
# Moran's I value
mi_pov.I

In [ ]:
# Calculate the p-value (confidence level)
mi_pov.p_sim

---
### *Exercise*
6. (7 points) Calculate Moran's I for Young Population (under 17 years old) in `census_gdf` column. Then, assign the result to `mi_age`. <br>
In addition, assign the actual Index to `mi_age_I` and p-value to `mi_age_p`. 

    Help: https://pysal.org/esda/generated/esda.Moran.html#esda.Moran

```python

    mi_age = esda.moran.Moran(`DATA FRAME`[`COLUMN NAME`], `SPATIAL WEIGHTS MATRIX`)
    mi_age_I = `CALL APPROPRIATE ATTRIBUTE`
    mi_age_p = `CALL APPROPRIATE ATTRIBUTE`
    print(f'Moran\'s I: {mi_age_I}, p-value: {mi_age_p}')
    
```
---

In [ ]:
# Your code here
mi_age = esda.moran.Moran(`DATA FRAME`[`COLUMN NAME`], `SPATIAL WEIGHTS MATRIX`)
mi_age_I = `CALL APPROPRIATE ATTRIBUTE`
mi_age_p = `CALL APPROPRIATE ATTRIBUTE`
print(f'Moran\'s I: {mi_age_I}, p-value: {mi_age_p}')

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(mi_age_I, 3) == 0.412
assert round(mi_age_p, 3) == 0.001

print('Success!')

### 3.3. Calculate LISA

Syntax for LISA is as follows:
```python
    # LISA (Local Indicator of Spatial Association)
    lisa = esda.moran.Moran_Local(`VARIABLE`, `SPATIAL WEIGHTS MATRIX`)
```

Variable `lisa` has the following attributes:
* `Is` : Local Moran's I value
* `q` : Quadrant (HH, HL, LH, LL)
* `p_sim` : p-value

source: https://pysal.org/esda/generated/esda.Moran_Local.html#esda.Moran_Local

In [ ]:
# Calculate LISA for poverty
lisa_pov = esda.Moran_Local(census_gdf['POVERTY'], w)
lisa_pov

In [ ]:
# q indicate quandrant location 1 HH, 2 LH, 3 LL, 4 HL
lisa_pov.q

In [ ]:
census_gdf['lisa_pov_q'] = lisa_pov.q

In [ ]:
# p_sim is the p-value of each value
lisa_pov.p_sim

In [ ]:
census_gdf['lisa_pov_pval'] = lisa_pov.p_sim

In [ ]:
census_gdf.head(15)

In [ ]:
# Assign results of LISA into each census region
lisa_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

for idx, row in census_gdf.iterrows():
    if row['lisa_pov_pval'] < 0.05:
        census_gdf.loc[idx, f'lisa_pov'] = lisa_dict[row['lisa_pov_q']]
    else:
        census_gdf.loc[idx, f'lisa_pov'] = 'NS'

census_gdf.head(15)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

lisa_color = {'HH': '#FF0000', # Red
              'LH': '#FFC0CB', # Pink
              'LL': '#0000FF', # Blue
              'HL': '#87CEEB', # Skyblue
              'NS': '#f0f0f0'  # Light grey
             } 

census_gdf.loc[census_gdf['lisa_pov'] == 'NS'].plot(ax=ax, color=lisa_color['NS'])
census_gdf.loc[census_gdf['lisa_pov'] == 'HH'].plot(ax=ax, color=lisa_color['HH'])
census_gdf.loc[census_gdf['lisa_pov'] == 'LH'].plot(ax=ax, color=lisa_color['LH'])
census_gdf.loc[census_gdf['lisa_pov'] == 'LL'].plot(ax=ax, color=lisa_color['LL'])
census_gdf.loc[census_gdf['lisa_pov'] == 'HL'].plot(ax=ax, color=lisa_color['HL'])

ax.set_title('LISA for Poverty')
ax.set_axis_off()
plt.show()

---
### *Exercise*
7. (7 points) Calculate LISA for Young Population (under 17 years old) in `census_gdf` column. Then, assign the result to `lisa_age`. <br>
In addition, assign the quadrant information and p-value information to `lisa_age_q`, `lisa_age_pval` columms in the `census_gdf`, respectively.  

    Help: https://pysal.org/esda/generated/esda.Moran_Local.html#esda.Moran_Local

```python
    lisa_age = esda.moran.Moran_Local(`DATA FRAME`[`COLUMN NAME`], `SPATIAL WEIGHTS MATRIX`)
    census_gdf['lisa_age_q'] = `CALL APPROPRIATE ATTRIBUTE`
    census_gdf['lisa_age_pval'] = `CALL APPROPRIATE ATTRIBUTE`
```
---

In [ ]:
# Your code here
lisa_age = esda.moran.Moran_Local(`DATA FRAME`[`COLUMN NAME`], `SPATIAL WEIGHTS MATRIX`)
census_gdf['lisa_age_q'] = `CALL APPROPRIATE ATTRIBUTE`
census_gdf['lisa_age_pval'] = `CALL APPROPRIATE ATTRIBUTE`

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert 'lisa_age_q' in census_gdf.columns
assert 'lisa_age_pval' in census_gdf.columns
assert census_gdf.at[10, 'lisa_age_q'] == 3
assert census_gdf.at[10, 'lisa_age_pval'] == 0.003

print('Success!')

Upon the completion of the exercise 7, the following code will give you the same map as shown below. 

<div>
<img src="./image/age_local_moran.png" width="300"/>
</div>


In [ ]:
# Assign results of LISA into each census region
lisa_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

for idx, row in census_gdf.iterrows():
    if row['lisa_age_pval'] < 0.05:
        census_gdf.loc[idx, f'lisa_age'] = lisa_dict[row['lisa_age_q']]
    else:
        census_gdf.loc[idx, f'lisa_age'] = 'NS'

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for key,val in lisa_color.items():
    census_gdf.loc[census_gdf['lisa_age'] == key].plot(ax=ax, color=val)

ax.set_title('LISA for Age')
ax.set_axis_off()
plt.show()

## 4. Visualize the results of social area analysis

In [ ]:
def calculate_moran_and_lisa(_gdf, _col):

    # Get the spatial weight matrix (contiguity matrix)
    _w = libpysal.weights.Queen.from_dataframe(_gdf)

    # Calculate Moran's I
    _mi = esda.Moran(_gdf[_col], _w)
    print(f'{_col} | Moran\'s I: {round(_mi.I, 3)}, p-value: {_mi.p_sim}')

    _lisa = esda.Moran_Local(_gdf[_col], _w)
    _gdf[f'lisa_{_col}_q'] = _lisa.q
    _gdf[f'lisa_{_col}_pval'] = _lisa.p_sim

    lisa_dict = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}
    
    lisa_color = {'HH': '#FF0000', # Red
                'LH': '#FFC0CB', # Pink
                'LL': '#0000FF', # Blue
                'HL': '#87CEEB', # Skyblue
                'NS': '#f0f0f0'  # Light grey
                } 
    
    for idx, row in _gdf.iterrows():
        if row[f'lisa_{_col}_pval'] < 0.05:
            _gdf.loc[idx, f'lisa_{_col}'] = lisa_dict[row[f'lisa_{_col}_q']]
        else:
            _gdf.loc[idx, f'lisa_{_col}'] = 'NS'

    return _gdf


for col in ['POVERTY', 'YOUNG', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN']:
    census_gdf = calculate_moran_and_lisa(census_gdf, col)

census_gdf.head(3)

In [ ]:
def lisa_dissolve(_gdf, _col):
    _lisa_plot = _gdf.dissolve(by=f"lisa_{_col}")
    _lisa_plot = _lisa_plot.reset_index()

    return _lisa_plot

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(['POVERTY', 'YOUNG', 'WHITE', 'BLACK', 'HISPANIC', 'ASIAN']):
    census_gdf.plot(col, cmap='Greys', scheme='NaturalBreaks', ax=axes[i])
    lisa_plot = lisa_dissolve(census_gdf, col)
    lisa_plot.loc[lisa_plot[f'lisa_{col}'] == 'HH'].boundary.plot(color='Red', ax=axes[i], linewidth=1, hatch='///')
    lisa_plot.loc[lisa_plot[f'lisa_{col}'] == 'LL'].boundary.plot(color='Blue', ax=axes[i], linewidth=1, hatch='///')

    axes[i].set_axis_off()
    axes[i].set_title(f'LISA for {col}')

plt.tight_layout()
plt.show()

# DONE